# Using `pymldb` Tutorial


Interactions with MLDB occurs via a [REST API](/doc#builtin/WorkingWithRest.md.html). Interacting with a REST API over HTTP from a Notebook interface can be a little bit laborious if you're using a general-purpose Python library like [`requests`](http://docs.python-requests.org/en/latest/) directly, so MLDB comes with a Python library called [`pymldb`](https://github.com/datacratic/pymldb) to ease the pain.


## Connections

The [`pymldb` library](https://github.com/datacratic/pymldb) includes a class called `Connection`. The recommended usage pattern is shown here:

In [1]:
from pymldb import Connection
mldb = Connection("http://localhost")

## Accessing the REST API

Once you have a connection object, you can access any resource in the [REST API](/doc#builtin/WorkingWithRest.md.html) under the `v1` attribute, which is a `Resource` object. A `Resource` object is just an extremely cheap-to-create, immutable proxy for a single URL (which may or may not exist). 

**Note**: Creating a `Resource` object does not result in any HTTP requests (see below).

In [2]:
print mldb.v1

http://localhost/v1


You can use a `Resource` object to quickly create new `Resource` objects to refer to different URLs by calling functions or passing in arguments, chaining the calls:

In [3]:
print type(mldb.v1), mldb.v1
x = mldb.v1.x
print type(x), x
y = x("y")
print type(y), y
z = mldb.v1("and").so("on")("and").so.on
print type(z), z

<class 'pymldb.resource.Resource'> http://localhost/v1
<class 'pymldb.resource.Resource'> http://localhost/v1/x
<class 'pymldb.resource.Resource'> http://localhost/v1/x/y
<class 'pymldb.resource.Resource'> http://localhost/v1/and/so/on/and/so/on


### Making HTTP requests

Once you have a `Resource` object that refers to a URL you care about, you can use it to issue HTTP requests:

In [4]:
dataset_types = mldb.v1.types.datasets
dataset_types.get()

<Response [200]>

In [5]:
#keyword arguments to get() are appended to the GET query string

mldb.v1.types.get(x="y")

<Response [200]>

In [6]:
#dictionaries arguments to put() and post() are sent as JSON via PUT or POST

mldb.v1.datasets("sample").put( {"type": "beh.mutable"} ) 

<Response [201]>

Here we create a dataset and insert two rows of two columns into it:

In [7]:
demo = mldb.v1.datasets("demo")
demo.put({"type":"beh.mutable"})
demo.rows.post({"rowName": "first", "columns":[["a",1,0],["b",2,0]]})
demo.rows.post({"rowName": "second", "columns":[["a",3,0],["b",4,0]]})
demo.commit.post({})

<Response [200]>

------

## SQL Queries

Now that we have a dataset, we can use the `query()` method on the connection to run an [SQL query](/doc/#builtin/sql/Sql.md.html) and get the results back as a Pandas DataFrame:

In [8]:
df = mldb.query("select * from demo")
print type(df)

<class 'pandas.core.frame.DataFrame'>


In [9]:
df

,a,b
_rowName,,
second,3,4
first,1,2


## Errors

`pymldb` HTTP requests will raise an exception if the status code returned by the server is not in the 2xx or 3xx range (i.e. if there is a problem with the request or if the server encounters an error while processing the request). This behaviour can be avoided by setting the argument `raise_on_error` to `False`.

In [10]:
bad_url = mldb.v1.this.url.does("not").exist
print bad_url

http://localhost/v1/this/url/does/not/exist


In [11]:
bad_url.get(raise_on_error=False)

<Response [404]>

In [12]:
bad_url.get()

ResourceError: '404 Not Found' response to 'GET http://localhost/v1/this/url/does/not/exist'

"unknown resource GET /v1/this/url/does/not/exist"

## Where to next?

Check out the other [Tutorials and Demos](/doc/#builtin/Demos.md.html).